In [1]:
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
import re
import time
from tqdm.notebook import tqdm, trange
import tqdm.notebook as tq

## Creating a dataframe of urls to request using game names

In [2]:
games_hmf_csv=(r'C:/Users/Nick/Desktop/GitHub/video-games-benchmark-with-custom-parts-master/Data/test/hmf_games.csv')
games_hmf_df=pd.read_csv(games_hmf_csv)

## Loading CPUs and GPUs to request as parameters

In [3]:
gpu_csv = (r'C:/Users/Nick/Desktop/GitHub/video-games-benchmark-with-custom-parts-master/Data/test/hmf_gpu.csv')
cpu_csv = (r'C:/Users/Nick/Desktop/GitHub/video-games-benchmark-with-custom-parts-master/Data/test/hmf_cpu.csv')
gpu_df=pd.read_csv(gpu_csv)
cpu_df=pd.read_csv(cpu_csv)

## Requesting

#### Creating the main dataframe to store our information

In [4]:
benchmark_df=pd.DataFrame({"Game" : pd.Series(dtype='str'),
                           "game_id" : pd.Series(dtype='str'),
                           "CPU" : pd.Series(dtype='str'),
                           "cpu_id" : pd.Series(dtype='str'),
                           "GPU" : pd.Series(dtype='str'),
                           "gpu_id" : pd.Series(dtype='str'),
                           "Low" : pd.Series(dtype='int'),  
                           "Medium" : pd.Series(dtype='int'),  
                           "High" : pd.Series(dtype='int'),  
                           "Ultra" : pd.Series(dtype='int'),})
url = "https://api.howmanyfps.com"
headers={"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"}

#### Our main function. Total requests needed to complete the 4 nested loops: 4 Graphic Settings * 102 GPUs * 167 CPUs * 20 Games makes a total of 1,362,720 requests.

In [1]:
game_level = list(range(20))
cpu_level = list(range(167))
gpu_level = list(range(10))
graphics_level = list(range(4))
graphics = ['Low', 'Medium', 'High', 'Ultra']
counter=1
for games in tq.tqdm(game_level, desc='Games', position=0, leave=False):
    for some in tq.tqdm(cpu_level, desc='CPUs', position=0, leave=False):
        for number in tq.tqdm(gpu_level, desc='GPUs', position=0, leave=False):
            for graphic in tq.tqdm(graphics_level, desc='Graphics', position=0, leave=False):
                time.sleep(0.01)
            
                data = {"gameId" : games_hmf_df['id'].iloc[games],
                        "presetName": graphics[graphic],"resolutionName":"1920x1080",
                        "cpuId":cpu_df['id'].iloc[some],
                        "gpuId":gpu_df['id'].iloc[number]}
                response_low = requests.post(url, json = data, headers=headers)
                fps=int(re.search(r'\d+', response_low.text).group())
                
                benchmark_df.loc[counter,:]=[games_hmf_df['Name'].iloc[games],
                                            games_hmf_df['id'].iloc[games],
                                            cpu_df['model'].iloc[some],
                                            cpu_df['id'].iloc[some],
                                            gpu_df['model'].iloc[number],
                                            gpu_df['id'].iloc[number],
                                            fps,
                                            fps,
                                            fps,
                                            fps]
                counter=counter+1

Medium


NameError: name 'tq' is not defined

#### Converting our float values to int

In [6]:
benchmark_df.Low = benchmark_df.Low.astype(int)
benchmark_df.Medium = benchmark_df.Medium.astype(int)
benchmark_df.High = benchmark_df.High.astype(int)
benchmark_df.Ultra = benchmark_df.Ultra.astype(int)

#### Exporting the data to our csv/excel file.

In [7]:
benchmark_df.to_csv('C:/Users/Nick/Desktop/GitHub/video-games-benchmark-with-custom-parts-master/Data/benchmark_1.csv', index=False)
benchmark_df.to_excel('C:/Users/Nick/Desktop/GitHub/video-games-benchmark-with-custom-parts-master/Data/benchmark_1.xlsx', index=False)